In [1]:
import pandas as pd
import numpy as np

In [81]:
df1 = pd.read_csv('/content/drive/MyDrive/오징어게임_imdb.csv')
df1 = df1.rename(columns={'review':'reviews'})
len(df1)

5172

In [3]:
df2 = pd.read_csv('/content/drive/MyDrive/오징어게임_my.csv')
df2 = df2[['Unnamed: 0', 'title', 'reviews']]

In [4]:
df3 = pd.read_csv('/content/drive/MyDrive/reviews_csv/Squid_Game_letterboxd.csv')

In [5]:
df = pd.concat([df1, df2, df3])

In [103]:
reviews = df['reviews'].to_list()
len(reviews)

8412

In [9]:
pip install langdetect

     |████████████████████████████████| 981 kB 4.3 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=b5938c082e5c02359f2a16dd0e26f3384b31ccf0d70fba838b55686b61f85fcf
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [10]:
from langdetect import detect

In [12]:
from tqdm import tqdm
import pickle
import csv
import re
import pandas as pd
import numpy as np
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.tokenize import TreebankWordTokenizer
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize

In [104]:
sentences = []

for i in reviews:
    text = re.sub('[^a-zA-Z\']',' ',i).strip()
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
    # text = re.sub(' +', ' ', text)
    text = ' '.join(text.split())
    sentences.append(text)
len(sentences)

8412

In [105]:
sample_list = [v for v in sentences if v] #공백제거

In [120]:
detect_list = []
for i in sample_list:
    if detect(i) == 'en':
        detect_list.append(i)

In [142]:
len(detect_list)

7087

In [126]:
from tqdm import tqdm
import pickle
import csv
import re
import pandas as pd
import numpy as np
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.tokenize import TreebankWordTokenizer
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize

In [110]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [111]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [112]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [118]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [127]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

In [128]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    elif pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

In [134]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, a):
        a = ''.join(a)
        word_tokens = self.tagger(a)
        
        words = []

        for i in word_tokens:
            text = re.sub('[^a-zA-Z0-9\']','',i).strip()
            text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
            if(text != ''):
                words.append(text)
        
        tag_words = nltk.pos_tag(words)
        # pos_words = [word for word in tag_words if word[1][0] in {'V','N','J','R'}]
        pos_words = []
        for word in tag_words:
            if word[1] in ['NNP', 'NN']:  #'VB', 'VBP', 'JJ'
                pos_words.append(word)
        temp_list = []
        for token, pos_tag in pos_words:
            tag = get_wordnet_pos(pos_tag)
            if tag != None:
                temp_list.append((token, get_wordnet_pos(pos_tag)))
        lemma = WordNetLemmatizer()
        token_final = [lemma.lemmatize(token, pos=tag) for token, tag in temp_list]
        long_words = [i for i in token_final if len(i) > 2]
        results = [w for w in long_words if w not in stop_words]
        return results

In [135]:
lda_tokens = []
custom_tokenizer = CustomTokenizer(text_to_word_sequence)
for sentences in detect_list:
    lda_tokens.append(custom_tokenizer(sentences))
lda_tokens[0]

['character', 'room', 'man', 'hair', 'board', 'plane', 'beginning']

In [141]:
len(lda_tokens)

7087

In [136]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [137]:
dictionary = corpora.Dictionary(lda_tokens)

2022-04-20 11:38:24,080 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-04-20 11:38:24,440 : INFO : built Dictionary(7398 unique tokens: ['beginning', 'board', 'character', 'hair', 'man']...) from 7087 documents (total 86432 corpus positions)


In [138]:
dictionary.filter_extremes(no_below=2, no_above=0.5)

2022-04-20 11:38:32,145 : INFO : discarding 3827 tokens: [('fluent', 1), ('flak', 1), ('steal', 1), ('apocalyptic', 1), ('blog', 1), ('denial', 1), ('enforcement', 1), ('grinding', 1), ('hault', 1), ('implausibility', 1)]...
2022-04-20 11:38:32,159 : INFO : keeping 3571 tokens which were in no less than 2 and no more than 3543 (=50.0%) documents
2022-04-20 11:38:32,177 : INFO : resulting dictionary: Dictionary(3571 unique tokens: ['beginning', 'board', 'character', 'hair', 'man']...)


In [139]:
corpus = [dictionary.doc2bow(text) for text in lda_tokens]

In [140]:
num_topics = 4
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=1
)

2022-04-20 11:38:52,351 : INFO : using autotuned alpha, starting with [0.25, 0.25, 0.25, 0.25]
2022-04-20 11:38:52,359 : INFO : using serial LDA version on this node
2022-04-20 11:38:52,365 : INFO : running online (multi-pass) LDA training, 4 topics, 20 passes over the supplied corpus of 7087 documents, updating model once every 2000 documents, evaluating perplexity every 2000 documents, iterating 400x with a convergence threshold of 0.001000
2022-04-20 11:38:55,693 : INFO : -8.860 per-word bound, 464.7 perplexity estimate based on a held-out corpus of 2000 documents with 32006 words
2022-04-20 11:38:55,695 : INFO : PROGRESS: pass 0, at document #2000/7087
2022-04-20 11:38:58,043 : INFO : optimized alpha [0.21964718, 0.13237226, 0.15311474, 0.10700433]
2022-04-20 11:38:58,045 : INFO : merging changes from 2000 documents into a model of 7087 documents
2022-04-20 11:38:58,055 : INFO : topic #0 (0.220): 0.052*"show" + 0.044*"series" + 0.037*"game" + 0.018*"season" + 0.017*"episode" + 0.01

In [143]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-04-20 11:41:48,767 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-04-20 11:41:48,783 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-04-20 11:41:48,794 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-04-20 11:41:48,820 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-04-20 11:41:48,838 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-04-20 11:41:48,856 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-04-20 11:41:48,872 : INFO : CorpusAccumulator accumulated stats from 7000 documents


Average topic coherence: -2.0088.
[([(0.049869336, 'game'),
   (0.029661233, 'show'),
   (0.016320908, 'money'),
   (0.013409186, 'series'),
   (0.013310249, 'drama'),
   (0.012550373, 'life'),
   (0.01157219, 'character'),
   (0.010133566, 'time'),
   (0.009911917, 'plot'),
   (0.00977579, 'episode'),
   (0.009015745, 'world'),
   (0.008750161, 'season'),
   (0.008674067, 'way'),
   (0.008483243, 'story'),
   (0.0077845487, 'end'),
   (0.006760865, 'death'),
   (0.006034615, 'capitalism'),
   (0.006025208, 'lot'),
   (0.0055228197, 'man'),
   (0.0053318264, 'nothing')],
  -1.7671865740292434),
 ([(0.056793477, 'series'),
   (0.04856806, 'show'),
   (0.032148268, 'episode'),
   (0.024977747, 'game'),
   (0.02329046, 'story'),
   (0.017084459, 'season'),
   (0.014696519, 'lot'),
   (0.013632986, 'plot'),
   (0.013380735, 'way'),
   (0.012896638, 'hype'),
   (0.011847112, 'time'),
   (0.011152911, 'something'),
   (0.0109744975, 'bit'),
   (0.010554547, 'character'),
   (0.010182724, 'en

In [144]:
%%capture
!pip install pyldavis

In [145]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

2022-04-20 11:42:24,286 : INFO : Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
2022-04-20 11:42:24,314 : INFO : Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [148]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
